In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler, LabelEncoder 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [4]:
#loading data
print("Cargar modelo...")
EMBEDD = "/Users/morenx/Downloads/mt/arxiv_embeddings.npy"
LABEL = "/Users/morenx/Downloads/mt/arxiv_labels.csv"

X = np.load(EMBEDD)
df_label = pd.read_csv(LABEL)
y_macro = df_label["macro_cat"]


#data size
print(f"Features (X): {X.shape}")
print(f"Etiquetas (y): {df_label.shape}")

Cargar modelo...
Features (X): (174703, 768)
Etiquetas (y): (174703, 2)


In [7]:
TEST_SIZE = 0.2
RANDOM_STATE = 42
#FASE 1 : MODELO A (MACRO CATEGORIAS)
y_macro = df_label["macro_cat"]

#datasplit
X_train, X_test, y_train, y_test = train_test_split(
    X,y_macro, test_size = TEST_SIZE, 
    random_state = RANDOM_STATE,
    stratify= y_macro
)
print()
print("Modelo A (Regresión Logística)...")

#logistic regression resulta una buena practica para  embeddings de alta dimension
model_A = LogisticRegression(
    solver="lbfgs",
    multi_class="multinomial",
    class_weight="balanced",
    max_iter=1000,
    n_jobs=-1,
    verbose=1
)
#train
model_A.fit(X_train, y_train)
print("\nRESULTADOS MODELO A ")
y_pred = model_A.predict(X_test)
print(classification_report(y_test, y_pred))



Modelo A (Regresión Logística)...


/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Exception ignored in: <function ResourceTracker.__del__ at 0x107171bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x1034bdbc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line


RESULTADOS MODELO A 
                        precision    recall  f1-score   support

      Computer Science       0.78      0.67      0.72      4000
             Economics       0.32      0.67      0.43       352
Electrical Engineering       0.71      0.84      0.77      2831
           Mathematics       0.88      0.85      0.87      4000
               Physics       0.93      0.91      0.92      4000
  Quantitative Biology       0.89      0.90      0.90      3977
  Quantitative Finance       0.85      0.87      0.86      1549
            Statistics       0.79      0.76      0.78      4000

              accuracy                           0.82     24709
             macro avg       0.77      0.81      0.78     24709
          weighted avg       0.83      0.82      0.82     24709



Como se puede notar en el reporte de clasificacion, contamos con un desbalanceo de clases...

Teniendo en cuenta que support imprime cuantas muestras hay en el conjunto de prueba para toda clase.
(Economics, EE, Quant Bio, Quant Fin, Statistics) que son las clases minoritarias son las clases que peor clasificacion tiene

DE NUEVO ME DIO MALOS RESULTADOS, A CONTINUACION COMENZARE A PROBAR DISTINTOS ALGORITMOS Y TECNICAS HASTA ENCONTRAR LO QUE SE ADECUE MEJOR A NUESTRAS NECESIDADES.

In [16]:
### mlp
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [17]:
# escalar datos
scaler = StandardScaler()
# ajustar el set de entrenamiento
X_train_scaled = scaler.fit_transform(X_train_downsampled) 
# solo aplicamos al set de prueba
X_test_scaled = scaler.transform(X_test)

In [18]:
#train
print("Modelo A con MLP...")
# hidden_layer_sizes: Dos capas ocultas (300 y 150 neuronas)
mlp_model = MLPClassifier(
    hidden_layer_sizes=(300, 150), 
    activation='relu', # Función de activación no lineal
    solver='adam',     # Optimizador estándar y eficiente
    max_iter=500,      # Debería ser suficiente para converger
    random_state=42,
    verbose=False
)
## fit
mlp_model.fit(X_train_scaled, y_train_downsampled)

# prediccion
y_pred_mlp = mlp_model.predict(X_test_scaled)

print("\nRESULTADOS MODELO A UNDER-SAMPLED + MLP")
print(classification_report(y_test, y_pred_mlp))

Modelo A con MLP...

RESULTADOS MODELO A UNDER-SAMPLED + MLP
                        precision    recall  f1-score   support

      Computer Science       0.83      0.80      0.81      4881
             Economics       0.23      0.23      0.23        35
Electrical Engineering       0.25      0.58      0.35       283
           Mathematics       0.86      0.88      0.87      7806
               Physics       0.98      0.92      0.95     20796
  Quantitative Biology       0.46      0.85      0.59       398
  Quantitative Finance       0.53      0.79      0.64       155
            Statistics       0.41      0.69      0.51       587

              accuracy                           0.89     34941
             macro avg       0.57      0.72      0.62     34941
          weighted avg       0.91      0.89      0.89     34941



NUEVA GENERACION DE EMBEDDINGS, DE NUEVO CON SENTENCETRANSFORMER, SOLO QUE EL DATASET QUE USE AHORA TIENE OTRA DISTRIBUCION DE CLASES

In [3]:
sampled = pd.read_csv("/Users/morenx/Downloads/mt/sampled.csv")

In [4]:
print(sampled['macro_cat'].value_counts())


macro_cat
Computer Science          20000
Mathematics               20000
Physics                   20000
Statistics                20000
Quantitative Biology      19886
Electrical Engineering    14153
Quantitative Finance       7743
Economics                  1761
Name: count, dtype: int64


In [8]:
#loading data
print("cargando modelo...")
EMBEDD2 = "/Users/morenx/Downloads/mt/arxiv_embeddings_sampled.npy"
LABEL2 = "/Users/morenx/Downloads/mt/arxiv_labels2.csv"

X = np.load(EMBEDD2)
df_label = pd.read_csv(LABEL2)
y_macro = df_label["macro_cat"]


#data size
print(f"Features (X): {X.shape}")
print(f"Etiquetas (y): {df_label.shape}")

cargando modelo...
Features (X): (123543, 768)
Etiquetas (y): (123543, 2)


In [12]:
##data split
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y_macro,
    test_size = TEST_SIZE,
    random_state = RANDOM_STATE,
    stratify = y_macro
)
print(f"entrenando con {X_train.shape[0]} registros....")
##logistic regression
clf = LogisticRegression(max_iter=1000, 
                        n_jobs=-1,
                        random_state=RANDOM_STATE)

##train
clf.fit(X_train, y_train)
##pred
y_pred =clf.predict(X_test)

print(" resultados... ")
print(f"Accuracy Global: {accuracy_score(y_test, y_pred):.4f}\n")
print(classification_report(y_test, y_pred))

entrenando con 98834 registros....


/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


 resultados... 
Accuracy Global: 0.8302

                        precision    recall  f1-score   support

      Computer Science       0.76      0.70      0.73      4000
             Economics       0.56      0.41      0.47       352
Electrical Engineering       0.74      0.80      0.77      2831
           Mathematics       0.88      0.85      0.87      4000
               Physics       0.92      0.91      0.92      4000
  Quantitative Biology       0.89      0.91      0.90      3977
  Quantitative Finance       0.87      0.90      0.88      1549
            Statistics       0.77      0.81      0.79      4000

              accuracy                           0.83     24709
             macro avg       0.80      0.79      0.79     24709
          weighted avg       0.83      0.83      0.83     24709



Exception ignored in: <function ResourceTracker.__del__ at 0x1034a1bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x107179bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x1031c5bc0>
Traceback (most recent call last

PRIMER BUEN RESULTADO!!!
aun asi, tenemos cosas que mejorar aun, la clase "ECONOMICS" tiene pocosdatos de soporte (352), AGREGARE EL PARAMETRO "class_weight" para buscar mejorar el recall de economics, aunque eso signifique sacrificar un poco de precision global...

si los resultados no me convencen, pasare a usar una mlp.


In [14]:
print(f"entrenando con {X_train.shape[0]} registros....")
##logistic regression
clf = LogisticRegression(max_iter=1000, 
                        n_jobs=-1,
                        class_weight="balanced",
                        random_state=RANDOM_STATE)

##train
clf.fit(X_train, y_train)
##pred
y_pred =clf.predict(X_test)

print(" resultados... ")
print(f"Accuracy Global: {accuracy_score(y_test, y_pred):.4f}\n")
print(classification_report(y_test, y_pred))

entrenando con 98834 registros....


/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


 resultados... 
Accuracy Global: 0.8209

                        precision    recall  f1-score   support

      Computer Science       0.78      0.67      0.72      4000
             Economics       0.32      0.67      0.43       352
Electrical Engineering       0.71      0.84      0.77      2831
           Mathematics       0.88      0.85      0.87      4000
               Physics       0.93      0.91      0.92      4000
  Quantitative Biology       0.89      0.90      0.90      3977
  Quantitative Finance       0.85      0.87      0.86      1549
            Statistics       0.79      0.76      0.78      4000

              accuracy                           0.82     24709
             macro avg       0.77      0.81      0.78     24709
          weighted avg       0.83      0.82      0.82     24709



Exception ignored in: <function ResourceTracker.__del__ at 0x102f9dbc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x105181bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x107a49bc0>
Traceback (most recent call last

Economics       0.56      0.41      0.47       352  
Economics "balanced"      0.32      0.67      0.43       352
Teniendo en cuenta que solo subio el recall, pero bajaron el f1 y la precision, aparte tambien bajo de rendimiento computer science y electrical engineering, considero que este es el limite de lo que una linea recta puede alcanzar. me ire a un mlp, teniendo en cuenta que tenemos una cantidad suficiente de datos.

investigando sobre la construccion de una mlp, lo puedo hacer con scikit o keras, la cosa es que scikit no tiene el parametro "class_weight=balanced", a continuacion realizare ambas opciones. 
teniendo en cuenta nuestras necesidades, pienso que servira la opcion de keras, aun asi intentare ambas.
en el peor de los casos, eliminare la categoria Economics.

In [18]:
from sklearn.preprocessing import LabelEncoder

print("Codificando etiquetas a números...")

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

Codificando etiquetas a números...


In [20]:
## mlp
print(" mlp training")

# early_stopping servira para parar el procedimiento si no hay mejora
mlp = MLPClassifier(
    hidden_layer_sizes=(256, 128),  #capas ocultas, una de 256 neuronas y otra de 128
    activation='relu',
    solver='adam',
    alpha=0.0001,      # Regularización L2 
    batch_size=32,     
    learning_rate_init=0.001,
    max_iter=50,       # epocas 
    early_stopping=True,
    validation_fraction=0.1, # Usa 10% para validar internamente
    random_state=42,
    verbose=True       
)

# train
mlp.fit(X_train, y_train_encoded)
# pred
y_pred_encoded = mlp.predict(X_test)

print("\n--- Resultados MLP (Scikit-learn) ---")
print(classification_report(y_test_encoded, y_pred_encoded, target_names=le.classes_))

 mlp training
Iteration 1, loss = 0.55218540
Validation score: 0.825071
Iteration 2, loss = 0.49211088
Validation score: 0.820518
Iteration 3, loss = 0.47160368
Validation score: 0.815662
Iteration 4, loss = 0.45463430
Validation score: 0.825981
Iteration 5, loss = 0.44116049
Validation score: 0.830939
Iteration 6, loss = 0.42695338
Validation score: 0.826588
Iteration 7, loss = 0.41465050
Validation score: 0.829017
Iteration 8, loss = 0.40447953
Validation score: 0.829826
Iteration 9, loss = 0.39053860
Validation score: 0.830635
Iteration 10, loss = 0.38017015
Validation score: 0.828106
Iteration 11, loss = 0.36799775
Validation score: 0.820720
Iteration 12, loss = 0.35777771
Validation score: 0.830028
Iteration 13, loss = 0.34862336
Validation score: 0.823452
Iteration 14, loss = 0.33746388
Validation score: 0.827499
Iteration 15, loss = 0.32754649
Validation score: 0.826791
Iteration 16, loss = 0.31922033
Validation score: 0.821934
Validation score did not improve more than tol=0.00

como no nos podemos librar de la clase minoritaria Economics.
Le agregare muestras sinteticas para que su numero de soporte.
teniendo en cuenta que los embeddings de specter son de alta calidad. el espacio es continuo, la tecnica que usare (smote) crea datos sinteticos (no titulo, no autor, pero SI SIGNIFICADO SEMANTICO), este checara los vecinos mas cercanos de papers en economics para crear mas datos. Tengo confianza que puede funcionar.
Por cada paper real de economics, tendremos 3 sinteticos, veamos como funciona


In [25]:
from imblearn.over_sampling import SMOTE
from collections import Counter
#codificamos economics, smote funciona con numeros codificados, no nombres
eco_label_index = le.transform(['Economics'])[0] 
print(f"La clase 'Economics' corresponde al número: {eco_label_index}")
### la estrategia sera hacer llegar a economics a 5000, las demas se quedaran igual
strategy = {
    eco_label_index: 5000 
}
#aplicar estrategia a smote
smote = SMOTE(sampling_strategy=strategy, random_state=RANDOM_STATE)

La clase 'Economics' corresponde al número: 1


In [26]:
print(f"Distribución antes: {Counter(y_train_encoded)}")

X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train_encoded)

print(f"Distribución después: {Counter(y_train_resampled)}")

Distribución antes: Counter({np.int64(0): 16000, np.int64(3): 16000, np.int64(7): 16000, np.int64(4): 16000, np.int64(5): 15909, np.int64(2): 11322, np.int64(6): 6194, np.int64(1): 1409})
Distribución después: Counter({np.int64(0): 16000, np.int64(3): 16000, np.int64(7): 16000, np.int64(4): 16000, np.int64(5): 15909, np.int64(2): 11322, np.int64(6): 6194, np.int64(1): 5000})


volvemos a evaluar nuestros modelos con estos datos sinteticos

In [27]:
# train
mlp.fit(X_train_resampled, y_train_resampled)
# pred
y_pred_encoded = mlp.predict(X_test)

print("\n--- Resultados MLP (Scikit-learn) ---")
print(classification_report(y_test_encoded, y_pred_encoded, target_names=le.classes_))

Iteration 1, loss = 0.56771120
Validation score: 0.813531
Iteration 2, loss = 0.50290124
Validation score: 0.818901
Iteration 3, loss = 0.47729956
Validation score: 0.826809
Iteration 4, loss = 0.45755582
Validation score: 0.816460
Iteration 5, loss = 0.43945571
Validation score: 0.825247
Iteration 6, loss = 0.42464405
Validation score: 0.826028
Iteration 7, loss = 0.40849257
Validation score: 0.826320
Iteration 8, loss = 0.39423521
Validation score: 0.828859
Iteration 9, loss = 0.38081647
Validation score: 0.827980
Iteration 10, loss = 0.36790068
Validation score: 0.832666
Iteration 11, loss = 0.35337234
Validation score: 0.830518
Iteration 12, loss = 0.34189807
Validation score: 0.832569
Iteration 13, loss = 0.33155072
Validation score: 0.827492
Iteration 14, loss = 0.32062477
Validation score: 0.830128
Iteration 15, loss = 0.30922816
Validation score: 0.822415
Iteration 16, loss = 0.30203011
Validation score: 0.825637
Iteration 17, loss = 0.28953131
Validation score: 0.818901
Iterat

regresion logistica con estos datos cientificos

In [30]:
lr = LogisticRegression(max_iter=1000, 
                        n_jobs=-1,
                        class_weight="balanced",
                        random_state=RANDOM_STATE)

##train
lr.fit(X_train_resampled, y_train_resampled)
##pred
y_pred =lr.predict(X_test)

print(" resultados... ")
print(f"Accuracy Global: {accuracy_score(y_test, y_pred):.4f}\n")
print(classification_report(y_test_encoded, y_pred, target_names=le.classes_))

/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


 resultados... 
Accuracy Global: 0.0000

                        precision    recall  f1-score   support

      Computer Science       0.78      0.67      0.72      4000
             Economics       0.34      0.65      0.45       352
Electrical Engineering       0.71      0.85      0.77      2831
           Mathematics       0.88      0.85      0.87      4000
               Physics       0.93      0.91      0.92      4000
  Quantitative Biology       0.89      0.90      0.90      3977
  Quantitative Finance       0.85      0.88      0.86      1549
            Statistics       0.79      0.76      0.78      4000

              accuracy                           0.82     24709
             macro avg       0.77      0.81      0.78     24709
          weighted avg       0.83      0.82      0.82     24709



Exception ignored in: <function ResourceTracker.__del__ at 0x102c39bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x1070e9bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x106ff5bc0>
Traceback (most recent call last

ESTA TECNICA DE LAS MUESTRAS SINTETICAS NO ME SIRVIO.
EN EL PROXIMO .IPYNB APLICARE LA TECNICA FEW SHOT LEARN. 